In [ ]:
import numpy as np
import pandas as pd

# ============================================================
# ① 周期的波形のサンプルデータを生成
# ============================================================
def generate_periodic_sample(
    fs: float = 2000.0,
    duration_s: float = 5.0,
    freqs_hz=(50.0, 120.0),
    amps=(1.0, 0.6),
    phases_rad=None,
    dc_offset: float = 0.0,
    noise_std: float = 0.1,
    n_sensors: int = 3,
    sensor_scale=None,
    sensor_phase_shift_rad=None,
    seed: int = 0,
    time_col: str = "t",
    sensor_prefix: str = "s",
) -> pd.DataFrame:
    """
    周期信号（複数正弦の和）+ DC + ノイズ を生成して DataFrame で返す。
    出力列: [time_col, s1, s2, ...]
    """
    rng = np.random.default_rng(seed)

    freqs_hz = np.asarray(freqs_hz, dtype=float)
    amps = np.asarray(amps, dtype=float)
    if phases_rad is None:
        phases_rad = np.zeros_like(freqs_hz, dtype=float)
    phases_rad = np.asarray(phases_rad, dtype=float)

    if sensor_scale is None:
        sensor_scale = np.ones(n_sensors, dtype=float)
    sensor_scale = np.asarray(sensor_scale, dtype=float)

    if sensor_phase_shift_rad is None:
        sensor_phase_shift_rad = np.zeros(n_sensors, dtype=float)
    sensor_phase_shift_rad = np.asarray(sensor_phase_shift_rad, dtype=float)

    N = int(round(fs * duration_s))
    t = np.arange(N, dtype=float) / fs

    # 基本波形（1本）
    base = np.full(N, dc_offset, dtype=float)
    for f, a, ph in zip(freqs_hz, amps, phases_rad):
        base += a * np.sin(2.0 * np.pi * f * t + ph)

    data = {time_col: t}
    for i in range(n_sensors):
        if sensor_phase_shift_rad[i] == 0.0:
            sig = sensor_scale[i] * base
        else:
            sig = np.full(N, dc_offset, dtype=float)
            for f, a, ph in zip(freqs_hz, amps, phases_rad):
                sig += sensor_scale[i] * a * np.sin(2.0 * np.pi * f * t + ph + sensor_phase_shift_rad[i])

        sig = sig + noise_std * rng.normal(size=N)
        data[f"{sensor_prefix}{i+1}"] = sig

    return pd.DataFrame(data)


# ============================================================
# ② 連続データを任意の窓ごとに「1行1窓」に変換（wide DataFrame）
# ============================================================
def to_window_wide_df(
    df: pd.DataFrame,
    sensor_cols,
    win_len: int,
    step: int,
    time_col: str = "t",
    include_t_center: bool = True,
) -> pd.DataFrame:
    """
    df の連続データを 1行=1窓 の wide DataFrame に変換して返す。
    列名: s1_000, s1_001, ..., s2_000, ...
    先頭に t_center 列を付ける（include_t_center=True の場合）。
    """
    if isinstance(sensor_cols, str):
        sensor_cols = [sensor_cols]

    x = df[sensor_cols].to_numpy(dtype=float)  # (N, S)
    N, S = x.shape

    if time_col in df.columns:
        t = df[time_col].to_numpy(dtype=float)
    else:
        t = np.arange(N, dtype=float)

    if N < win_len:
        raise ValueError("データ長 N が win_len より短いです。")

    starts = np.arange(0, N - win_len + 1, step, dtype=int)
    n_win = len(starts)

    # wide配列（sensor-major：s1全部→s2全部→...）
    wide = np.empty((n_win, win_len * S), dtype=float)
    if include_t_center:
        t_centers = np.empty(n_win, dtype=float)

    half = (win_len - 1) / 2.0
    for i, st in enumerate(starts):
        seg = x[st:st + win_len, :]          # (win_len, S)
        wide[i, :] = seg.T.reshape(-1)       # (S*win_len,)
        if include_t_center:
            t_centers[i] = t[st + int(round(half))]

    cols = []
    for s in sensor_cols:
        cols += [f"{s}_{k:03d}" for k in range(win_len)]

    wide_df = pd.DataFrame(wide, columns=cols)
    if include_t_center:
        wide_df.insert(0, "t_center", t_centers)

    return wide_df


# ============================================================
# ②の wide DataFrame を「(n_win, L, S)」に戻す（③で使用）
# ============================================================
def wide_df_to_windows(
    wide_df: pd.DataFrame,
    sensor_cols,
    win_len: int,
    t_center_col: str = "t_center",
):
    """
    1行=1窓の wide_df から windows 配列 (n_win, win_len, S) を復元する。
    t_center 列があれば t_centers も返す（無ければ None）。
    """
    if isinstance(sensor_cols, str):
        sensor_cols = [sensor_cols]

    n_win = wide_df.shape[0]
    S = len(sensor_cols)

    windows = np.empty((n_win, win_len, S), dtype=float)

    for si, sname in enumerate(sensor_cols):
        cols = [f"{sname}_{k:03d}" for k in range(win_len)]
        missing = [c for c in cols if c not in wide_df.columns]
        if missing:
            raise ValueError(f"wide_df に必要な列がありません（例）: {missing[:3]}")
        windows[:, :, si] = wide_df[cols].to_numpy(dtype=float)

    t_centers = None
    if t_center_col in wide_df.columns:
        t_centers = wide_df[t_center_col].to_numpy(dtype=float)

    return windows, t_centers


# ============================================================
# ③ windows をFFT特徴量へ（DataFrameで返す・metaは作らない）
# ============================================================
def fft_features_from_windows_df(
    windows: np.ndarray,              # (n_win, L, S)
    fs: float | None,                 # fs不明なら None（周波数は正規化: cycles/sample）
    k: int = 20,                      # DC除外で 1..k を特徴にする
    use_hann: bool = True,
    amp_correction: str = "hann",     # "none" / "n" / "hann"
    feature_mode: str = "amplitude",  # "amplitude" / "power"
    use_log1p: bool = True,
    add_mean: bool = True,
    sensor_names=None,                # 例: ["s1","s2","s3"]（Noneなら s0,s1,...）
):
    """
    window配列からFFT特徴量 DataFrame を作る（metaは返さない）。

    戻り値:
      X_df : (n_win, n_features) の DataFrame
      freq : rfftの周波数軸
             - fs指定: Hz
             - fs=None: cycles/sample（正規化周波数）
    列名の例（add_mean=True, feature_mode="amplitude"）:
      s1_mean, s1_amp_f001, ..., s1_amp_f020, s2_mean, s2_amp_f001, ...
    """
    if windows.ndim != 3:
        raise ValueError("windows は (n_win, L, S) の3次元配列を想定しています。")

    n_win, L, S = windows.shape
    if k < 1 or k > (L // 2):
        raise ValueError("k は 1 以上かつ L//2 以下にしてください（rfftの範囲内）。")

    if sensor_names is None:
        sensor_names = [f"s{i}" for i in range(S)]
    if len(sensor_names) != S:
        raise ValueError("sensor_names の長さが windows のセンサ数 S と一致しません。")

    # 窓関数
    w = np.hanning(L).astype(float) if use_hann else np.ones(L, dtype=float)
    w_sum = float(np.sum(w))

    # 周波数軸
    if fs is None:
        freq = np.fft.rfftfreq(L, d=1.0)          # cycles/sample
        freq_unit = "cyc"
    else:
        freq = np.fft.rfftfreq(L, d=1.0 / fs)     # Hz
        freq_unit = "hz"

    # FFT
    xw = windows * w[None, :, None]
    X = np.fft.rfft(xw, axis=1)
    mag = np.abs(X)

    # 振幅補正（特徴量としての目安）
    if amp_correction == "none":
        amp = mag
    elif amp_correction == "n":
        amp = (2.0 / L) * mag
    elif amp_correction == "hann":
        amp = (2.0 / w_sum) * mag
    else:
        raise ValueError("amp_correction は 'none' / 'n' / 'hann' のいずれかです。")

    # 特徴モード
    if feature_mode == "amplitude":
        core0 = amp
        kind = "amp"
    elif feature_mode == "power":
        core0 = amp ** 2
        kind = "pow"
    else:
        raise ValueError("feature_mode は 'amplitude' / 'power' のいずれかです。")

    # DC除外で 1..k
    core = core0[:, 1:k+1, :]  # (n_win, k, S)

    if use_log1p:
        core = np.log1p(core)
        kind = f"log1p_{kind}"

    # DataFrame化（センサごとに [mean, f001..fK] を並べる）
    cols = []
    blocks = []

    for si, sname in enumerate(sensor_names):
        if add_mean:
            mu = np.mean(windows[:, :, si], axis=1, keepdims=True)  # (n_win, 1)
            blocks.append(mu)
            cols.append(f"{sname}_mean")

        blocks.append(core[:, :, si])  # (n_win, k)
        for b in range(1, k + 1):
            # 「何番目のビンか」を列名に残す（周波数値を列名に埋めない＝fs違いでも安全）
            # 必要なら freq[b] を使って別に参照できる
            cols.append(f"{sname}_{kind}_{freq_unit}{b:03d}")

    X_feat = np.concatenate(blocks, axis=1)
    X_df = pd.DataFrame(X_feat, columns=cols)

    return X_df, freq



In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# ============================================================
# 使い方（ここだけ触ればOK）
# ============================================================

# 1) 対象CSVを「フォルダ内から1つ選ぶ」
DATA_DIR = Path("data")           # このスクリプト/ノートと同階層のフォルダ名
CSV_NAME = "anomaly-free.csv"     # フォルダ内のCSVファイル名（これを選ぶ）

# 2) 列名の指定
TIME_COL   = "t"                 # 時刻列名（無いなら下の処理でサンプル番号を使う）
SENSOR_COLS = ["s1", "s2", "s3"]  # FFT特徴量化したいセンサ列

# 3) 窓化（連続時系列 → 1行=1窓）
WIN  = 256
# WIN: 1窓に含める点数（例: 256点）
#   - 大きいほど：周波数分解能が細かい（Δf = fs/WIN が小さくなる）が、変化に鈍くなる/窓数が減る
#   - 小さいほど：変化に敏感になるが、周波数分解能は粗くなる

STEP = 256
# STEP: 窓のずらし幅
#   - STEP=WIN   → 窓が重ならない（非重複）
#   - STEP=WIN/2 → 50%重複（窓数が増え、時系列に戻したとき滑らか。ただし計算量増）

INCLUDE_T_CENTER = True
# INCLUDE_T_CENTER: 各窓の代表時刻 t_center を付けるか
#   - True 推奨（後で特徴量を時系列に戻してプロットしやすい）

# 4) FFT特徴量の設定
FS = 2000.0
# FS: サンプリング周波数[Hz]（1秒あたりのサンプル数）
#   - 周波数軸(Hz)の計算に必須
#   - 周波数分解能は Δf = FS / WIN
#   - 例: 2000/256 ≈ 7.8125 Hz刻み

K = 30
# K: DC(0Hz)を除いた「1..K番目の周波数ビン」を特徴量にする
#   - Kを増やすほど高周波まで見る（高周波異常に敏感）一方、特徴量次元が増えて学習が不安定になりやすい
#   - 制約: 1 <= K <= WIN//2

USE_HANN = True
# USE_HANN: 窓関数（ハニング）を掛けるか
#   - True推奨：窓の端の切れ目由来の“余計な周波数成分”を減らし、特徴量のブレを抑える

AMP_CORRECTION = "none"
# AMP_CORRECTION: FFT振幅スケールの“目安”補正
#   - "none": 補正なし（比較・学習が目的ならこれでもOK。設定を固定することが重要）
#   - "n"   : (2/WIN)*|FFT| の簡易補正（窓関数なし前提）
#   - "hann": (2/sum(hann))*|FFT| の簡易補正（USE_HANN=Trueと整合）

FEATURE_MODE = "power"
# FEATURE_MODE:
#   - "amplitude": 振幅（大きさ）
#   - "power": 振幅^2（大きい成分をより強く効かせたいとき）

USE_LOG1P = True
# USE_LOG1P: log(1+x) を取るか
#   - True推奨：極端に大きい成分の影響を抑え、学習を安定させやすい

ADD_MEAN = True
# ADD_MEAN: 各窓の「時間領域の平均」を追加するか（上下シフト検知に効く）
#   - DC(0Hz)成分は窓関数の影響で歪みやすいので、平均は別途入れる設計

# 5) 出力（任意）
SAVE_OUT = True
OUT_DIR  = DATA_DIR.parent / "out_fft_features"


# ============================================================
# 実行（ここから下は基本触らない）
# ============================================================

# 対象CSVのパス
csv_path = DATA_DIR / CSV_NAME
if not csv_path.exists():
    raise FileNotFoundError(f"CSVが見つかりません: {csv_path}")

# CSV読込（encodingが怪しい場合は、あなたの _read_csv_robust を使うのが安全）
df = pd.read_csv(csv_path)

# 時刻列が無い場合はサンプル番号を時刻として追加（窓中心 t_center を作るため）
if TIME_COL not in df.columns:
    df = df.copy()
    df[TIME_COL] = np.arange(len(df), dtype=float)

# センサ列の存在チェック
missing = [c for c in SENSOR_COLS if c not in df.columns]
if missing:
    raise ValueError(f"CSVに必要なセンサ列がありません: {missing} / file={csv_path}")

# ① 連続時系列 → 1行=1窓（wide化）
wide_df = to_window_wide_df(
    df,
    sensor_cols=SENSOR_COLS,
    win_len=WIN,
    step=STEP,
    time_col=TIME_COL,
    include_t_center=INCLUDE_T_CENTER
)

# ② wide → windows配列に復元（FFTへ渡す形 (n_win, WIN, S)）
windows, t_centers = wide_df_to_windows(
    wide_df,
    sensor_cols=SENSOR_COLS,
    win_len=WIN,
    t_center_col="t_center"
)

# ③ windows → FFT特徴量（DataFrame）
X_feat, freq = fft_features_from_windows_df(
    windows,
    fs=FS,                   # 周波数軸をHzで出すために必須
    k=K,                     # DC除外で1..K
    use_hann=USE_HANN,       # 窓関数
    amp_correction=AMP_CORRECTION,
    feature_mode=FEATURE_MODE,
    use_log1p=USE_LOG1P,
    add_mean=ADD_MEAN,       # 窓平均（上下シフト用）
    sensor_names=SENSOR_COLS # 列名接頭辞を元データと一致させる
)

print("df:", df.shape)
print("wide_df:", wide_df.shape)       # (窓数, 1 + センサ数*WIN)  ※t_center込み
print("windows:", windows.shape)       # (窓数, WIN, センサ数)
print("X_feat:", X_feat.shape)         # (窓数, センサ数*(K + mean有無))
print("X_feat columns head:", X_feat.columns[:12].tolist())

# ④ 保存（任意）
if SAVE_OUT:
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    X_out = X_feat.copy()
    # 追跡用メタ情報（後で「何窓目か」「元のどの時刻か」が分かる）
    X_out.insert(0, "win_i", np.arange(len(X_out), dtype=int))
    X_out.insert(1, "t_center", t_centers if t_centers is not None else np.nan)
    X_out.insert(2, "source_file", CSV_NAME)

    X_out.to_csv(OUT_DIR / f"{Path(CSV_NAME).stem}_fftfeat.csv", index=False)

    # 周波数ビン対応表（bin番号→Hz）を保存
    pd.DataFrame({
        "bin": np.arange(len(freq), dtype=int),
        "freq_hz": freq.astype(float),
    }).to_csv(OUT_DIR / "freq_bins.csv", index=False)
